In [12]:
import pandas as pd
import numpy as np

# Read in files

In [13]:
train = pd.read_csv("data/train.csv")
print(train.shape)
train.head()

(200, 8)


,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


Read in sample question and multiple choice options

In [14]:
i = 0
print("Prompt is: ", train.loc[i, "prompt"])
print("-----------------------")
for option in ["A","B","C","D","E"]:
    print("Option ",option,": ",train.loc[i,option])

Prompt is:  Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?
-----------------------
Option  A :  MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
Option  B :  MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
Option  C :  MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
Option  D :  MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a fac

# OpenAI installation and import of API

To use gpt-3.5-turbo, required to buy tokens on OpenAI API site. https://platform.openai.com/account/usage \
gpt-3.5-turbo is the predecessor to gpt-4 model, so am using this for now (to use gpt-4, monthly subscription is required)

In [15]:
import sys
!{sys.executable} -m pip install openai

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
import os
import openai


# Loop through questions and have GPT predict answer to each one

In [18]:
answer_list=[]

# func to go through all questions and enter it into gpt
for i in range(train.shape[0]):
    prompt=train.loc[i,"prompt"]    
    prompt_str='prompt question is: '+prompt+'\\n----------------------------------\\n'
    for option in ["A","B","C","D","E"]:
        prompt_str=prompt_str+'Option ' + option + ' : ' + train.loc[i,option] + '\\n'    

    # returns gpt's response of what it thinks correct answer is
    response=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "user", "content": prompt_str},
            {"role": "user", "content": 'Please select the most accurate option from the choices A to E above and answer just like "The answer is A".'}
        ],
    # temperature is a mark of creativity (0 is direct; 1 is most creative). for this purpose (science test), we want gpt to be straightforward
    temperature =0
    )
    print(response["choices"][0]["message"]["content"])
    # add each answer in given order to test accuracy at the end
    answer_list.append(response["choices"][0]["message"]["content"][14:15])
print("done")

The answer is D.
The answer is A.
The answer is C.
The answer is A.
The answer is B.
The answer is B.
The answer is C.
The answer is B.
The answer is B: Redshifting.
The answer is A.
The answer is E.
The answer is A.
The answer is E.
The answer is E.
The answer is B.
The answer is D.
The answer is E: Bernoulli's principle.
The answer is E.
The answer is A.
The answer is E.
The answer is D.
The answer is C.
The answer is C.
The answer is C.
The answer is D: 0.782 MeV.
The answer is E.
The answer is A.
The answer is D.
The answer is B.
The answer is C.
The answer is B.
The answer is E: Transfer function.
The answer is D.
The answer is D.
The answer is A.
The answer is B.
The answer is A.
The answer is A.
The answer is E.
The answer is A.
The answer is E.
The answer is A.
The answer is D: Vogel and Scheiner.
The answer is A.
The answer is A.
The answer is A.
The answer is B: Makoto Kobayashi and Toshihide Maskawa.
The answer is C.
The answer is D.
The answer is B.
The answer is B.
The ans

Write out csv of correct answers vs. gpt prediction for each question

In [20]:
train["prediction"]=answer_list
train.to_csv("./prediction.csv")

In [22]:
train[['answer', 'prediction']]

,answer,prediction
0,D,D
1,A,A
2,A,C
3,C,A
4,D,B
...,...,...
195,C,C
196,B,C
197,B,A
198,D,D


Use accuracy score as a metric to score gpt-3.5's performance on the exam

In [23]:
from sklearn.metrics import accuracy_score
print("accuracy score is : ",accuracy_score(train["answer"],train["prediction"]))

accuracy score is :  0.68


Accuracy score for the science exam is 68% by gpt-3.5-turbo model. That's great for a first run, but would like to see how gpt-4 performs and future iterations.